In [1]:
#imports
import pandas as pd
import requests
import openai
import json
import csv
import os
import time
from create_csv import main as create_csv #usando alias para não ocupar o nome main
from dotenv import load_dotenv

In [2]:
#automatizando a criação do csv
create_csv()

In [5]:
#api url
sdw2023_api_url = 'https://sdw-2023-prd.up.railway.app/users'

#carregando a secret key pelo dotenv
load_dotenv()

openai.api_key = os.getenv('KEY')

df = pd.read_csv('../doc/SDW2023.csv')
user_ids = df['UserID'].tolist()
print(user_ids)

#Por questões de praticidade alterei os 5 primeiros ids para os 5 primeiro usuarios
sdw2023_user_ids = user_ids[-1]
print(sdw2023_user_ids)

[1, 2, 3, 4, 5]
5


In [6]:
#extract
#Na data não existe usuarios com id 2,3,4 ... Estou utilizando os 5 primeiros usuarios de /users

def get_user(api_url, qtd_user_ids):

    try:
        response = requests.get(api_url)
        response.raise_for_status()

        users_data = response.json()
        #validando os usuarios, alguns usuarios não tem nome por exemplo. Melhorando a qualidade dos dados
        valid_users = [user for user in users_data if user.get('id') is not None and user.get('name') is not None]

        return valid_users[:qtd_user_ids]
    
    except Exception as e:
        print(f"Erro {str(e)}")
        return None
    



In [7]:

users_list = get_user(sdw2023_api_url, sdw2023_user_ids)

print(json.dumps(users_list, indent=2))

[
  {
    "id": 1,
    "name": "Devweekerson",
    "account": {
      "id": 1,
      "number": "01.097954-4",
      "agency": "2030",
      "balance": 624.12,
      "limit": 1000.0
    },
    "card": {
      "id": 1,
      "number": "xxxx xxxx xxxx 1111",
      "limit": 2000.0
    },
    "features": [
      {
        "id": 1,
        "icon": "https://digitalinnovationone.github.io/santander-dev-week-2023-api/icons/pix.svg",
        "description": "PIX"
      },
      {
        "id": 2,
        "icon": "https://digitalinnovationone.github.io/santander-dev-week-2023-api/icons/pay.svg",
        "description": "Pagar"
      },
      {
        "id": 3,
        "icon": "https://digitalinnovationone.github.io/santander-dev-week-2023-api/icons/transfer.svg",
        "description": "Transferir"
      },
      {
        "id": 4,
        "icon": "https://digitalinnovationone.github.io/santander-dev-week-2023-api/icons/account.svg",
        "description": "Conta Corrente"
      },
      {
        

In [11]:
#Transform

def generate_ai_news(user):
    completion = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {
                "role": "system",
                "content": "Você é um especialista em marketing bancário."
            },
            {
                "role": "user",
                "content": f"Crie uma mensagem para {user['name']} sobre a importância dos investimentos (máximo de 100 caracteres)"
            }
        ]
    )
    
    time.sleep(21) #sleep para o rate limit do chatgpt: 3 / min
    return completion.choices[0].message.content.strip('\"')

for user in users_list:
  news = generate_ai_news(user)
  print(news)
  user['news'].append({
      "icon": "https://digitalinnovationone.github.io/santander-dev-week-2023-api/icons/credit.svg",
      "description": news
  })


Invista no seu futuro financeiro!
Invista hoje, colha amanhã.
Invista agora, colha amanhã.
Invista no seu futuro financeiro!
Invista no seu futuro financeiro!


In [ ]:
def update_user(user):
  response = requests.put(f"{sdw2023_api_url}/users/{user['id']}", json=user)
  return True if response.status_code == 200 else False

for user in users_list:
  success = update_user(user)
  print(f"User {user['name']} updated? {success}!")